# Demonstrație Algoritmul Bernstein-Vazirani

Algoritmul Bernstein-Vazirani este un algoritm cuantic care determină un șir secret într-o singură interogare a unui oracol, în timp ce algoritmul clasic necesită n interogări.

## Problema

Dată o funcție f: {0,1}ⁿ → {0,1} de forma:
$$f(x) = s \cdot x \mod 2$$

unde s este un șir secret de n biți și · reprezintă produsul scalar.

**Obiectiv**: Găsirea șirului secret s

**Clasic**: n interogări (câte una pentru fiecare bit)
**Cuantic**: 1 interogare!

In [ ]:
# Importuri necesare
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, circuit_drawer
import matplotlib.pyplot as plt
from itertools import product

print("Librării importate cu succes!")

## Fundamentele Matematice

### Funcția Bernstein-Vazirani
Pentru un șir secret s = s₁s₂...sₙ și input x = x₁x₂...xₙ:
$$f(x) = s \cdot x = (s_1 \cdot x_1) \oplus (s_2 \cdot x_2) \oplus ... \oplus (s_n \cdot x_n)$$

### Strategia Clasică
Pentru a găsi s clasic:
1. Testăm x = 100...0 → obținem s₁
2. Testăm x = 010...0 → obținem s₂
3. ...
4. Testăm x = 000...1 → obținem sₙ

Total: n evaluări

### Strategia Cuantică
1. Creăm superpoziție uniformă: $\frac{1}{\sqrt{2^n}} \sum_{x} |x\rangle$
2. Aplicăm oracolul: $|x\rangle|0\rangle \rightarrow |x\rangle|f(x)\rangle$
3. Folosim qubit auxiliar în starea |-⟩ pentru kick-back de fază
4. Aplicăm Hadamard pentru a extrage s direct

In [ ]:
def create_bernstein_vazirani_oracle(secret_string):
    """Creează oracolul pentru șirul secret dat"""
    n = len(secret_string)
    oracle = QuantumCircuit(n + 1, name=f'Oracle s={secret_string}')
    
    # Pentru fiecare bit din șirul secret
    for i, bit in enumerate(secret_string):
        if bit == '1':
            # Dacă bitul secret este 1, aplicăm CNOT
            # cu controlul pe qubit-ul i și target pe qubit-ul auxiliar (ultimul)
            oracle.cx(i, n)
    
    return oracle

def bernstein_vazirani_algorithm(secret_string):
    """Implementează algoritmul Bernstein-Vazirani"""
    n = len(secret_string)
    
    # Creăm circuitul: n qubiti pentru input + 1 qubit auxiliar
    qc = QuantumCircuit(n + 1, n)
    
    # Pasul 1: Inițializăm qubit-ul auxiliar în |1⟩
    qc.x(n)
    
    # Pasul 2: Aplicăm Hadamard pe toți qubiti
    for i in range(n + 1):
        qc.h(i)
    
    # Pasul 3: Aplicăm oracolul
    oracle = create_bernstein_vazirani_oracle(secret_string)
    qc = qc.compose(oracle)
    
    # Pasul 4: Aplicăm Hadamard pe primii n qubiti
    for i in range(n):
        qc.h(i)
    
    # Pasul 5: Măsurăm primii n qubiti
    for i in range(n):
        qc.measure(i, i)
    
    return qc

# Testăm cu diferite șiruri secrete
test_secrets = ['101', '1010', '11001', '101010']

for secret in test_secrets:
    print(f"\nȘirul secret: {secret}")
    oracle = create_bernstein_vazirani_oracle(secret)
    print(f"Oracolul pentru s={secret}:")
    print(oracle.draw())

In [ ]:
# Rulăm algoritmul pentru fiecare șir secret
simulator = AerSimulator()
shots = 1024

results = []

for secret in test_secrets:
    print(f"\n{'='*50}")
    print(f"Testăm șirul secret: {secret}")
    print(f"{'='*50}")
    
    # Creăm și rulăm circuitul
    qc = bernstein_vazirani_algorithm(secret)
    
    # Simulăm
    transpiled = transpile(qc, simulator)
    job = simulator.run(transpiled, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    # Găsim rezultatul cel mai frecvent
    most_frequent = max(counts, key=counts.get)
    confidence = counts[most_frequent] / shots * 100
    
    results.append((secret, qc, counts, most_frequent))
    
    print(f"Rezultate măsurătoare: {counts}")
    print(f"Șirul detectat: {most_frequent}")
    print(f"Confidența: {confidence:.1f}%")
    
    # Verificăm corectitudinea
    if most_frequent == secret:
        print("✅ SUCCESS! Șirul secret a fost găsit corect!")
    else:
        print("❌ ERROR! Șirul detectat nu corespunde cu cel secret.")
    
    # Afișăm circuitul pentru primul exemplu
    if secret == test_secrets[0]:
        print(f"\nCircuitul pentru s={secret}:")
        print(qc.draw())

In [ ]:
# Vizualizăm rezultatele
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, (secret, qc, counts, detected) in enumerate(results):
    ax = axes[i]
    
    # Pregătim datele - afișăm doar primele 8 rezultate pentru claritate
    sorted_counts = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True)[:8])
    
    labels = list(sorted_counts.keys())
    values = list(sorted_counts.values())
    
    # Colorăm diferit rezultatul corect
    colors = ['lightgreen' if label == secret else 'lightblue' for label in labels]
    
    bars = ax.bar(range(len(labels)), values, color=colors)
    ax.set_title(f'Secret: {secret}\nDetectat: {detected}', fontsize=12, weight='bold')
    ax.set_ylabel('Numărul de măsurători')
    ax.set_xlabel('Șiruri măsurate')
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha='right')
    
    # Adăugăm valorile pe bare
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{value}',
                ha='center', va='bottom', fontsize=8)
    
    # Indicator de succes
    success = detected == secret
    status_color = 'green' if success else 'red'
    status_text = '✅ SUCCES' if success else '❌ EȘEC'
    
    ax.text(0.02, 0.98, status_text, transform=ax.transAxes,
            fontsize=10, weight='bold', color=status_color,
            va='top', ha='left',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Statistici generale
total_tests = len(results)
successful_tests = sum(1 for _, _, _, detected in results if detected == results[i][0] for i in range(len(results)))
success_rate = successful_tests / total_tests * 100

print(f"\n{'='*60}")
print(f"STATISTICI GENERALE")
print(f"{'='*60}")
print(f"Teste totale: {total_tests}")
print(f"Teste reușite: {successful_tests}")
print(f"Rata de succes: {success_rate:.1f}%")
print(f"{'='*60}")

In [ ]:
def classical_bernstein_vazirani(secret_string, oracle_func):
    """Simulează abordarea clasică pentru problema Bernstein-Vazirani"""
    n = len(secret_string)
    discovered_secret = ['0'] * n
    evaluations = 0
    
    # Pentru fiecare poziție, testăm cu un șir care are 1 doar în acea poziție
    for i in range(n):
        test_input = ['0'] * n
        test_input[i] = '1'
        test_string = ''.join(test_input)
        
        # Evaluăm oracolul
        result = oracle_func(test_string)
        evaluations += 1
        
        # Rezultatul ne spune direct valoarea bit-ului secret
        discovered_secret[i] = str(result)
    
    return ''.join(discovered_secret), evaluations

def bernstein_vazirani_oracle_function(secret_string):
    """Funcția oracol pentru problema Bernstein-Vazirani"""
    def oracle(input_string):
        # Calculăm produsul scalar modulo 2
        result = 0
        for i in range(len(secret_string)):
            result ^= int(secret_string[i]) * int(input_string[i])
        return result
    return oracle

# Comparăm abordarea clasică cu cea cuantică
print("COMPARAȚIA CLASIC vs CUANTIC\n")
print(f"{'Secret':<10} {'Clasic':<10} {'Eval.':<5} {'Cuantic':<10} {'Eval.':<5} {'Avantaj':<10}")
print("-" * 70)

for secret in test_secrets:
    # Abordarea clasică
    oracle_func = bernstein_vazirani_oracle_function(secret)
    classical_result, classical_evals = classical_bernstein_vazirani(secret, oracle_func)
    
    # Abordarea cuantică (din rezultatele anterioare)
    quantum_result = next(detected for s, _, _, detected in results if s == secret)
    quantum_evals = 1
    
    # Calculăm avantajul
    advantage = f"{classical_evals}x"
    
    print(f"{secret:<10} {classical_result:<10} {classical_evals:<5} {quantum_result:<10} {quantum_evals:<5} {advantage:<10}")

print("\n" + "="*70)
print("AVANTAJUL CUANTIC:")
print(f"- Clasic: O(n) evaluări ale oracolului")
print(f"- Cuantic: O(1) evaluare a oracolului")
print(f"- Pentru n mari: Speedup exponențial!")
print("="*70)

In [ ]:
# Demonstrație pas cu pas pentru un exemplu simplu
simple_secret = '101'
n = len(simple_secret)

print(f"DEMONSTRAȚIE PAS CU PAS pentru secret s = {simple_secret}")
print("="*60)

# Pasul 1: Starea inițială
print("\nPasul 1: Starea inițială")
print(f"Qubiti de input: |000⟩")
print(f"Qubit auxiliar: |1⟩")
print(f"Starea totală: |000⟩ ⊗ |1⟩ = |0001⟩")

# Pasul 2: După aplicarea Hadamard
print("\nPasul 2: După aplicarea Hadamard pe toți qubiti")
print(f"Qubiti de input: H⊗3|000⟩ = (|0⟩+|1⟩)/√2 ⊗ (|0⟩+|1⟩)/√2 ⊗ (|0⟩+|1⟩)/√2")
print(f"                = (1/√8) Σ|x⟩ pentru x ∈ {{000, 001, 010, 011, 100, 101, 110, 111}}")
print(f"Qubit auxiliar: H|1⟩ = (|0⟩-|1⟩)/√2")

# Pasul 3: Efectul oracolului
print("\nPasul 3: Aplicarea oracolului pentru s = 101")
print(f"Oracolul calculează f(x) = s·x = s₁x₁ ⊕ s₂x₂ ⊕ s₃x₃")
print(f"Pentru s = 101: f(x) = 1·x₁ ⊕ 0·x₂ ⊕ 1·x₃ = x₁ ⊕ x₃")

# Calculăm efectul pentru fiecare stare de bază
print(f"\nEfectul asupra fiecărei stări:")
for x in ['000', '001', '010', '011', '100', '101', '110', '111']:
    # Calculăm f(x) = x₁ ⊕ x₃ pentru s = 101
    f_x = int(x[0]) ^ int(x[2])  # x₁ ⊕ x₃
    sign = '−' if f_x == 1 else '+'
    print(f"|{x}⟩ → ({sign}1)|{x}⟩ ⊗ (|0⟩-|1⟩)/√2")

print(f"\nDupă oracol, starea devine:")
print(f"(1/√8) [+|000⟩ − |001⟩ + |010⟩ − |011⟩ − |100⟩ + |101⟩ − |110⟩ + |111⟩] ⊗ (|0⟩-|1⟩)/√2")

# Pasul 4: După Hadamard final
print("\nPasul 4: După aplicarea Hadamard pe qubiti de input")
print(f"Transformarea Hadamard extrage șirul secret direct!")
print(f"Rezultatul măsurătorii va fi |{simple_secret}⟩ cu probabilitate 1")

# Verificăm prin simulare
qc_demo = bernstein_vazirani_algorithm(simple_secret)
transpiled_demo = transpile(qc_demo, simulator)
job_demo = simulator.run(transpiled_demo, shots=1000)
result_demo = job_demo.result()
counts_demo = result_demo.get_counts()

print(f"\nVerificare prin simulare:")
print(f"Rezultate: {counts_demo}")
most_frequent_demo = max(counts_demo, key=counts_demo.get)
print(f"Cel mai frecvent rezultat: {most_frequent_demo}")
print(f"Corectitudine: {'✅' if most_frequent_demo == simple_secret else '❌'}")

## Aplicații și Extensii

### Aplicații Practice

1. **Criptografie cuantică**: Detecția cheilor secrete în protocoale de comunicație

2. **Testarea funcțiilor**: Verificarea proprietăților funcțiilor booleene

3. **Algoritmi de căutare**: Baza pentru algoritmi mai complecși de căutare cuantică

### Extensii și Variante

1. **Bernstein-Vazirani recursiv**: Pentru funcții cu structură ierarhică

2. **Versiuni cu zgomot**: Adaptări pentru hardware cuantic real

3. **Implementări hibride**: Combinarea cu algoritmi clasici

### Limitări

1. **Specificitatea problemei**: Funcționează doar pentru funcții de forma f(x) = s·x

2. **Scalabilitatea**: Pentru hardware actual, limitarea vine de la coherența cuantică

3. **Zgomotul**: Erorile pot afecta corectitudinea rezultatului

In [ ]:
# Analiză de performanță pentru diferite dimensiuni
import time

def analyze_scaling():
    """Analizează scalarea algoritmului cu dimensiunea problemei"""
    sizes = [3, 4, 5, 6]  # Dimensiuni de testat
    
    print("ANALIZA SCALĂRII ALGORITMULUI\n")
    print(f"{'n':<3} {'Secret':<10} {'Clasic':<8} {'Cuantic':<8} {'Speedup':<8} {'Timp (ms)':<10}")
    print("-" * 55)
    
    for n in sizes:
        # Generăm un șir secret aleator
        secret = ''.join(np.random.choice(['0', '1'], n))
        
        # Măsurăm timpul pentru algoritmul cuantic
        start_time = time.time()
        qc = bernstein_vazirani_algorithm(secret)
        transpiled = transpile(qc, simulator)
        job = simulator.run(transpiled, shots=100)  # Mai puține shots pentru viteză
        result = job.result()
        end_time = time.time()
        
        quantum_time = (end_time - start_time) * 1000  # în milisecunde
        
        # Complexități teoretice
        classical_complexity = n
        quantum_complexity = 1
        speedup = classical_complexity / quantum_complexity
        
        print(f"{n:<3} {secret:<10} {classical_complexity:<8} {quantum_complexity:<8} {speedup:.1f}x{'':<3} {quantum_time:.2f}")
    
    print("\nNotă: Timpii măsurați includ overhead-ul de simulare")
    print("Pe hardware cuantic real, diferențele ar fi mai dramatice")

analyze_scaling()

## Concluzie

Algoritmul Bernstein-Vazirani demonstrează un avantaj cuantic clar și practic:

### Avantaje Cheie

1. **Speedup exponențial**: De la O(n) la O(1) evaluări ale oracolului
2. **Deterministic**: Rezultatul este garantat să fie corect (fără zgomot)
3. **Elegant**: Demonstrează puterea superpoziției și interferenței cuantice

### Principii Demonstrate

1. **Paralelism cuantic**: Evaluarea simultană pentru toate input-urile posibile
2. **Phase kickback**: Transferul informației prin faze cuantice
3. **Interferența constructivă**: Amplificarea rezultatului corect

### Importanța în Contextul Larger

- **Precursor pentru algoritmi complecși**: Shor, Grover, etc.
- **Proof of concept**: Demonstrație clară a avantajului cuantic
- **Fundament teoretic**: Pentru înțelegerea calculului cuantic

### Următorii Pași

1. Explorați algoritmul Simon pentru probleme de periodicitate
2. Studiați implementări pe hardware cuantic real
3. Investigați tehnici de corectare a erorilor pentru menținerea avantajului cuantic